<a href="https://colab.research.google.com/github/scottthomaswx/RadarHodographs/blob/main/Level3_RadarHodograph_MostRecent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Needed Packages

In [1]:
# @title
%%capture
!pip install matplotlib
!pip install numpy
!pip install arm-pyart
!pip install metpy
!pip install pint
!pip install math
!pip install os
!pip install pandas
!pip install warnings
!pip install datetime
!pip install requests
!pip install glob

import matplotlib.pyplot as plt
from metpy.units import units
import pyart
import numpy as np
from metpy.plots import Hodograph
import metpy.calc as mpcalc
import matplotlib.colors as colors
from metpy.io import Level3File
from pint import UnitRegistry
import math
import os
import pandas as pd
import warnings
from datetime import datetime, timedelta
import requests

!rm -rf sample_data
!mkdir hodos
!mkdir sr_hodos
!mkdir radar_files

#Give Needed Information

In [119]:
radar_id = 'KXYZ' #Make Uppercase
#Synoptic API Token... If you dont have one must use Manual (set winds manually) or None for Surface Winds
synoptic_token = '' #Enter Synoptic API Token. If one is not provided user must use Manual or None for Windspeed

#Get Storm Motion and Plot Info

In [120]:
#Storm Motion
storm_motion_method = 'Bunkers Right' #Choose Mean Wind, Bunkers Left, Bunkers Right, User Selected, Corfidi Downshear, Corfidi Upshear

#Enter User Selected Storm Motion If Applicable
sm_dir = 777
sm_speed = 77

#Surface Winds
#Preset or ASOS_ID Require Entering A Synoptic API Token Above

sfc_status = 'Preset' #Enter Preset, ASOS_ID, Manual, or None

#Plot Info

range_type = 'Static' #Enter Dynamic For Changing Range From Values or Static for Constant Range Value
static_value = 70 # Enter Static Hodo Range or 999 To Not Use

#For ASOS_ID
nearest_asos = 'KXYZ' #Enter nearest ASOS Station if ASOS_ID Selected

#Manual Entry Surface Wind if Manual Selected
sfc_dir = 777
sfc_spd = 77

#Pull VWP Data

In [121]:
# @title
url1 = f'https://tgftp.nws.noaa.gov/SL.us008001/DF.of/DC.radar/DS.48vwp/SI.{radar_id.lower()}/sn.last'
r1 = requests.get(url1)
f= f'/content/radar_files/latest_radar'
with open(f, 'wb') as f1:
      f1.write(r1.content)

#Pull VWP

In [122]:
# @title
L3 = Level3File(f)

#Get Meta Data
scan_time = L3.metadata['prod_time']

#Get Radar Data
data_table = L3.tab_pages

radar_height = L3.prod_desc.height

#Get Number Of Pages
page_list = []
for page in data_table:
  if "VAD Algorithm Output" in page:
    page_list.append(data_table.index(page))

#Put All Pages In One Table
vwp = []
for idx in page_list:
  if idx ==0 :
    strip_table_0 = data_table[idx].split('\n')
    vwp.extend(strip_table_0)

  elif idx >=1:
    strip_table = data_table[idx].split('\n')[3:]
    vwp.extend(strip_table)

  vwp_body = vwp [3:]

#Set Variable Lists
ALT = []
U_Wind = []
V_Wind = []
DIR = []
SPD = []

#Add Variables To Lists
for line in vwp_body:
  ALT.append(line.split()[0])
  U_Wind.append(line.split()[1])
  V_Wind.append(line.split()[2])
  DIR.append(line.split()[4])
  SPD.append(line.split()[5])

#Do Unit Conversions
ALT_m = []
for item in ALT:
  ALT_m.append(((float(item) * 100) - float(radar_height))  * 0.3048 )

U_Wind_Knots = []
for item in U_Wind:
  U_Wind_Knots.append(float(item) * 1.94384)

V_Wind_Knots = []
for item in V_Wind:
  V_Wind_Knots.append(float(item) * 1.94384)

ALT_m = np.array(ALT_m)
U_Wind_Knots = np.array(U_Wind_Knots)
V_Wind_Knots = np.array(V_Wind_Knots)

if scan_time.month < 10:
  month = '0' + str(scan_time.month)
else:
  month = str(scan_time.month)

if scan_time.day < 10:
  day = '0' + str(scan_time.day)
else:
  day = str(scan_time.day)

if scan_time.hour < 10:
  hour = '0' + str(scan_time.hour)
else:
  hour = str(scan_time.hour)

if scan_time.minute < 10:
  minute = '0' + str(scan_time.minute)
else:
  minute = str(scan_time.minute)

if scan_time.second < 10:
  second = '0' + str(scan_time.second)
else:
  second = str(scan_time.second)

#Pull Surface Winds

In [123]:
# @title
API_ROOT = "https://api.synopticdata.com/v2/"
API_TOKEN = synoptic_token

def mesowest_get_sfcwind(api_args):
    """
    For each station in a list of stations, retrieves all observational data
    within a defined time range using mesowest API. Writes the retrieved data
    and associated observation times to a destination file. API documentation:

        https://api.synopticdata.com/v2/stations/nearesttime

    Parameters
    ----------
      api_args  : dictionary


    Returns
    -------
        jas_ts  : json file
                dictionary of all observations for a given station.
                What is most significant, however, is writing the
                observed data to a file that then can be manipulated
                for plotting.

    """
    station = api_args["stid"]
    api_request_url = os.path.join(API_ROOT, "stations/nearesttime")
    req = requests.get(api_request_url, params=api_args)
    jas_ts = req.json()
    for s in range(0,len(jas_ts['STATION'])):
        try:
            station = jas_ts['STATION'][s]
            stn_id = station['STID']
            ob_times = station['OBSERVATIONS']['wind_speed_value_1']['date_time']
            wnspd = station['OBSERVATIONS']['wind_speed_value_1']['value']
            wndir = station['OBSERVATIONS']['wind_direction_value_1']['value']
        except:
            pass
    return wnspd, wndir

def calc_components(speed, direction):
  u_comp = speed * np.cos(np.deg2rad(direction))
  v_comp = speed * np.sin(np.deg2rad(direction))
  return u_comp, v_comp

def conv_angle_enter(ang):
  ang = 270 - ang
  if ang < 0:
    ang += 360
  if ang > 360:
    ang -= 360
  return ang

if sfc_status == 'ASOS_ID':
  api_args = {"token":API_TOKEN, "stid": f"{nearest_asos}", "attime": f"{scan_time.year}{month}{day}{hour}{minute}", "within": 60,"status":"active", "units":"speed|kts",  "hfmetars":'1'}
  wnspd, wndir = mesowest_get_sfcwind(api_args)
  sfc_dir = wndir
  sfc_spd = wnspd

if sfc_status == 'Preset':
  radar_list = pd.read_csv('https://raw.githubusercontent.com/scottthomaswx/RadarHodographs/main/RadarInfo.csv')
  track = np.where(radar_list['Site ID'] == radar_id)
  nearest_asos = radar_list['Primary ASOS'][track[0]].item()
  api_args = {"token":API_TOKEN, "stid": f"{nearest_asos}", "attime": f"{scan_time.year}{month}{day}{hour}{minute}", "within": 60,"status":"active", "units":"speed|kts",  "hfmetars":'1'}
  wnspd, wndir = mesowest_get_sfcwind(api_args)
  if wndir == ''or wnspd  == '':
    nearest_asos = radar_list['Secondary ASOS'][track[0]].item()
    newapi_args = {"token":API_TOKEN, "stid": f"{nearest_asos}", "attime": f"{scan_time.year}{month}{day}{hour}{minute}", "within": 60,"status":"active", "units":"speed|kts",  "hfmetars":'1'}
    wnspd, wndir = mesowest_get_sfcwind(newapi_args)
  sfc_dir = wndir
  sfc_spd = wnspd

if sfc_status == 'Manual':
  sfc_dir = sfc_dir
  sfc_spd = sfc_spd

if sfc_status == 'None':
  sfc_dir = 0
  sfc_spd = 0

##Calculate Sfc Wind Components
if sfc_dir != 'None':
  try:
    sfc_angle = conv_angle_enter(sfc_dir)
  except:
    sfc_angle = '---'
  sfc_u, sfc_v = calc_components(sfc_spd, sfc_angle)

  np.insert(U_Wind_Knots, 0, sfc_u)
  np.insert(V_Wind_Knots, 0, sfc_v)
  np.insert(ALT_m, 0, 0)

#Max Height Check And Interpolation
##Read Output To Check Restrictions on Storm Motion

In [124]:
# @title
max_height = float(ALT_m[len(ALT_m)-1])
if max_height < 6000:
  warnings.warn("\n \033[1mWARNING:\033[0m \n Max wind height is under 6000m AGL \n Must use User Selected Storm Motion to get Storm Relative Hodograph or SRH/SWV")
else:
  print("Max wind height is greater than 6000m AGL: Use Any Storm Motion")

#Interpolate To Key Heights

if max_height >= 300:
  u_300 = np.interp(500, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_300 = np.interp(500, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_300 = np.nan
  v_300 = np.nan

if max_height >= 500:
  u_500 = np.interp(500, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_500 = np.interp(500, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_500 = np.nan
  v_500 = np.nan

if max_height >= 1000:
  u_1000 = np.interp(1000, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_1000 = np.interp(1000, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_1000 = np.nan
  v_1000 = np.nan

if max_height >= 3000:
  u_3000 = np.interp(3000, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_3000 = np.interp(3000, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_3000 = np.nan
  v_3000 = np.nan

if max_height >= 6000:
  u_6000 = np.interp(6000, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_6000 = np.interp(6000, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_6000 = np.nan
  v_6000 = np.nan

if max_height >= 8000:
  u_8000 = np.interp(8000, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_8000 = np.interp(8000, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_8000 = np.nan
  v_8000 = np.nan

if max_height >= 9000:
  u_9000 = np.interp(9000, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_9000 = np.interp(9000, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_9000 = np.nan
  v_9000 = np.nan

if max_height >= 11000:
  u_11000 = np.interp(11000, ALT_m, U_Wind_Knots, left=np.nan, right=np.nan)
  v_11000 = np.interp(11000, ALT_m, V_Wind_Knots, left=np.nan, right=np.nan)
else:
  u_11000 = np.nan
  v_11000 = np.nan

#Calculate Storm Motion and Parameters



In [125]:
# @title
def calc_components(speed, direction):
  u_comp = speed * np.cos(np.deg2rad(direction))
  v_comp = speed * np.sin(np.deg2rad(direction))
  return u_comp, v_comp

def calc_vector(u_comp, v_comp):
  mag = np.sqrt(u_comp**2 + v_comp**2)
  dir = np.rad2deg(np.arctan2(u_comp, v_comp)) % 360
  return mag, dir

def calc_shear(u_layer, v_layer, u_bottom, v_bottom):
  u_shr = u_layer - u_bottom
  v_shr = v_layer - v_bottom
  shrmag = np.hypot(u_shr, v_shr)
  return shrmag


def calc_meanwind(u_wind, v_wind, layer_min, layer_max):
  layer_top = np.where(ALT_m >= layer_max)[0][0]
  layer_bottom = np.where(ALT_m >= layer_min)[0][0]

  front_val_u = uv_dict[str(layer_min)]["u"]
  front_val_v = uv_dict[str(layer_min)]["v"]

  back_val_u = uv_dict[str(layer_max)]["u"]
  back_val_v = uv_dict[str(layer_max)]["v"]

  u_meanwind = u_wind[layer_bottom:layer_top]
  v_meanwind = v_wind[layer_bottom:layer_top]

  if layer_min !=0:
    u_meanwind = np.insert(u_meanwind, 0, front_val_u )
    u_meanwind = np.insert(v_meanwind, 0, front_val_v)
  else:
    pass

  u_meanwind = np.append(u_meanwind, back_val_u)
  v_meanwind = np.append(v_meanwind, back_val_v)

  mean_u = np.mean(u_meanwind)
  mean_v = np.mean(v_meanwind)

  return mean_u, mean_v

def calc_bunkers():
  layer_top = np.where(ALT_m >= 6000)[0][0]
  u_meanwind = np.append(U_Wind_Knots[:layer_top], u_6000)
  v_meanwind = np.append(V_Wind_Knots[:layer_top], v_6000)
  mean_u = np.mean(u_meanwind)
  mean_v = np.mean(v_meanwind)

  u_shr = u_meanwind[-1] - U_Wind_Knots[0]
  v_shr = v_meanwind[-1] - U_Wind_Knots[0]

  dev = 7.5 * 1.94

  dev_amnt = dev / np.hypot(u_shr, v_shr)
  rstu = mean_u + (dev_amnt * v_shr)
  rstv = mean_v - (dev_amnt * u_shr)
  lstu = mean_u - (dev_amnt * v_shr)
  lstv = mean_v + (dev_amnt * u_shr)
  rmag, rdir = calc_vector(rstu, rstv)
  lmag, ldir = calc_vector(lstu, lstv)

  return rstu, rstv, lstu, lstv, rmag, rdir, lmag, ldir

def calc_corfidi(u_layer, v_layer, ALT_m, u_mean, v_mean):
  llj_top = np.where(ALT_m >= (1500))[0][0]
  llj_u = np.append(U_Wind_Knots[:llj_top], u_6000)
  llj_v = np.append(V_Wind_Knots[:llj_top], v_6000)

  mag, dir = calc_vector(llj_u, llj_v)
  max=0
  i=0
  for a in mag:
    if mag[i] >= mag[i-1]:
      max = i

  u_max = llj_u[i]
  v_max = llj_v[i]

  corfidi_up_u = u_mean - u_max
  corfidi_up_v =  v_mean - v_max

  corfidi_down_u = u_mean + corfidi_up_u
  corfidi_down_v = v_mean + corfidi_up_v

  return corfidi_up_u, corfidi_up_v, corfidi_down_u, corfidi_down_v

def conv_angle_param(ang):
  ang = ang+180
  if ang < 0:
    ang += 360
  if ang > 360:
    ang -= 360
  return ang

def conv_angle_enter(ang):
  ang = 270 - ang
  if ang < 0:
    ang += 360
  if ang > 360:
    ang -= 360
  return ang

def calc_dtm(u_300, v_300, rmu, rmv):
  dtm_u = rmu + u_300 /2
  dtm_v = rmv + v_300 /2
  return dtm_u, dtm_v

def calculate_sr_wind(u_wind, v_wind):
    try:
      sr_u = U_Wind_Knots - u_wind
      sr_v = V_Wind_Knots - v_wind
      sr_mw_u = u_mean - u_wind
      sr_br_u = rmu - u_wind
      sr_bl_u = lmu - u_wind
      sr_mw_v = v_mean - v_wind
      sr_br_v = rmv - v_wind
      sr_bl_v = lmv - v_wind
      sr_sfc_u = sfc_u - u_wind
      sr_sfc_v = sfc_v - v_wind
      sr_cu_u =  cvu_u - u_wind
      sr_cd_u = cvd_u - u_wind
      sr_cu_v = cvu_v - v_wind
      sr_cd_v = cvd_v - v_wind
      sr_dtm_u = dtm_u - u_wind
      sr_dtm_v = dtm_v - v_wind
      sr_sm_u = u_sm - u_wind
      sr_sm_v = v_sm - v_wind
      return sr_u, sr_v, sr_mw_u, sr_br_u, sr_bl_u, sr_mw_v, sr_br_v, sr_bl_v, sr_sfc_u, sr_sfc_v, sr_cu_u, sr_cd_u, sr_cd_v, sr_cu_v, sr_dtm_u, sr_dtm_v, sr_sm_u, sr_sm_v
    except:
      warnings.warn('ERROR: Data Missing For Storm Relative calculations with this method: For data missing under 6000m AGL User Selected Storm Motion Required')

uv_dict ={"0":{ "u":U_Wind_Knots[0], "v":V_Wind_Knots[0]},"300":{"u":u_300, "v": v_300}, "500":{"u":u_500,"v":v_500}, "1000":{"u":u_1000,"v":v_1000}, "3000":{"u":u_3000,"v":v_3000}, "6000":{"u":u_6000, "v":v_6000}, "8000":{"u":u_8000, "v":v_8000}, "9000":{"u":u_9000, "v":v_9000}, "11000":{ "u":u_11000, "v":v_11000}}

#Calculate Bulk Shear
if max_height >= 500:
  shr005 = calc_shear(u_500, v_500, U_Wind_Knots[0], V_Wind_Knots[0])
  if np.isnan(shr005) == True:
    shr005 = '--'
  else:
    shr005 = round(shr005)
else:
  shr005 = '--'

if max_height >= 1000:
  shr01 = calc_shear(u_1000, v_1000, U_Wind_Knots[0], V_Wind_Knots[0])
  if np.isnan(shr01):
    shr01= '--'
  else:
    shr01 = round(shr01)
else:
  shr01 = '--'

if max_height >= 3000:
  shr03 = calc_shear(u_3000, v_3000, U_Wind_Knots[0], V_Wind_Knots[0])
  shr13 = calc_shear(u_3000, v_1000, u_1000, v_1000)
  if np.isnan(shr03):
    shr03 = '--'
  else:
    shr03 = round(shr03)

  if np.isnan(shr13):
    shr13 = '--'
  else:
    shr13 = round(shr13)
else:
  shr03 = '--'
  shr13 = '--'

if max_height >= 6000:
  shr06 = calc_shear(u_6000, v_6000, U_Wind_Knots[0], V_Wind_Knots[0])
  shr36 = calc_shear(u_6000, v_6000, u_3000, v_3000)
  if np.isnan(shr06):
    shr06 = '--'
  else:
    shr06 = round(shr06)

  if np.isnan(shr36):
    shr36 = '--'
  else:
    shr36 = round(shr36)
else:
  shr06 = '--'
  shr36 = '--'

if max_height >= 8000:
  shr08 = calc_shear(u_8000, v_8000, U_Wind_Knots[0], V_Wind_Knots[0])
  shr68 = calc_shear(u_8000, v_8000, u_6000, v_6000)
  if np.isnan(shr08):
    shr08 = '--'
  else:
    shr08 = round(shr08)

  if np.isnan(shr68):
    shr68 = '--'
  else:
    shr68 = round(shr68)
else:
  shr08 = '--'
  shr68 = '--'

if max_height >= 11000:
  shr911 = calc_shear(u_11000, v_11000, u_9000, v_9000)
  if np.isnan(shr911):
    shr911 = '--'
  else:
    shr911 = round(shr911)
else:
  shr911 = '--'

#Calculate Storm Motions
if max_height >= 6000:
  u_mean, v_mean = calc_meanwind(U_Wind_Knots, V_Wind_Knots, 0,6000)
  mean_mag, mean_dir = calc_vector(u_mean, v_mean)
  if np.isnan(mean_mag) == False:
    mean_mag = round(mean_mag)
  if np.isnan(mean_mag):
    mean_mag = '--'
  rmu, rmv, lmu, lmv, rmag, rdir, lmag, ldir = calc_bunkers()
  if np.isnan(rmag) == False:
    rmag = round(rmag)
  if np.isnan == False:
    rmag = '--'
  if np.isnan(lmag) == False:
    lmag = round(lmag)
  if np.isnan(lmag):
    lmag = '--'
  cvu_u, cvu_v, cvd_u, cvd_v = calc_corfidi(U_Wind_Knots, V_Wind_Knots, ALT_m, u_mean, v_mean)
  cor_u_mag, cor_u_dir = calc_vector(cvu_u, cvu_v)
  if np.isnan(cor_u_mag) == False:
    cor_u_mag = round(cor_u_mag)
  if np.isnan(cor_u_mag):
    cor_u_mag = '--'
  cor_d_mag, cor_d_dir = calc_vector(cvd_u, cvd_v)
  if np.isnan(cor_d_mag) == False:
   cor_d_mag =  round(cor_d_mag)
  if np.isnan(cor_d_mag):
    cor_d_mag = '--'
else:
  mean_mag = '--'
  mean_dir = '---'
  rmu = np.nan
  rmv = np.nan
  lmu = np.nan
  lmv = np.nan
  rmag = '--'
  rdir = '---'
  lmag = '--'
  ldir = '---'
  cor_u_mag = '--'
  cor_u_dir = '---'
  cor_d_mag = '--'
  cor_d_dir = '---'
  u_mean = np.nan
  v_mean = np.nan
  rmu = np.nan
  rmv = np.nan
  lmu = np.nan
  lmv = np.nan
  cvu_u = np.nan
  cvu_v = np.nan
  cvd_u = np.nan
  cvd_v = np.nan

#Calculate Deviant Tornado Motion
if max_height >= 6000:
  u_300, v_300 = calc_meanwind(U_Wind_Knots, V_Wind_Knots, 0, 300)
  dtm_u, dtm_v = calc_dtm(u_300, v_300, rmu, rmv)
  dtm_mag, dtm_dir = calc_vector(dtm_u, dtm_v)
  if np.isnan(dtm_mag) == False:
   dtm_mag =  round(dtm_mag)
else:
  dtm_mag, dtm_dir = '--'
  dtm_u = np.nan
  dtm_v = np.nan

#Calculate meteorological angles
try:
  sfc_angle = conv_angle_enter(sfc_dir)
except:
  sfc_angle = '---'
try:
  us_ang_cor = conv_angle_enter(sm_dir)
except:
  us_ang_cor = '---'
try:
  mean_dirmet = conv_angle_param(mean_dir)
  if np.isnan(mean_dirmet) == False:
    mean_dirmet = round(mean_dirmet)
  if np.isnan(mean_dirmet):
    mean_dirmet = '---'
except:
  mean_dirmet = '---'
try:
  rang = conv_angle_param(rdir)
  if np.isnan(rang) == False:
   rang =  round(rang)
  if np.isnan(rang):
   rang =  '---'
except:
  rang = '---'
try:
  lang = conv_angle_param(ldir)
  if np.isnan(lang) == False:
   lang = round(lang)
  if np.isnan(lang):
   lang = '---'
except:
  lang = '---'
try:
  down_adj = conv_angle_param(cor_d_dir)
  if np.isnan(down_adj) == False:
    down_adj = round(down_adj)
  if np.isnan(down_adj):
    down_adj = '---'
except:
  down_adj = '---'
try:
  up_adj = conv_angle_param(cor_u_dir)
  if np.isnan(up_adj) == False:
   up_adj = round(up_adj)
  if np.isnan(up_adj):
   up_adj = '---'
except:
  up_adj = '---'
try:
  dtm_dir_cor = conv_angle_param(dtm_dir)
  if np.isnan(dtm_dir_cor) == False:
    dtm_dir_cor = round(dtm_dir_cor)
  if np.isnan(dtm_dir_cor):
    dtm_dir_cor = '---'
except:
  dtm_dir_cor = '---'


#Calculate Sfc Wind Components
if sfc_dir != 'None':
  try:
    sfc_angle = conv_angle_enter(sfc_dir)
  except:
    sfc_angle = '---'
  sfc_u, sfc_v = calc_components(sfc_spd, sfc_angle)
  if np.isnan(sfc_angle) == False:
    sfc_angle = round(sfc_angle)
  if np.isnan(sfc_angle):
    sfc_angle = '---'

  #Calculate User Selected Motion Components
  try:
    us_ang_cor = conv_angle_enter(sm_dir)
  except:
    us_ang_cor = '---'
  u_sm, v_sm = calc_components(sm_speed, us_ang_cor)
  if np.isnan(us_ang_cor) == False:
    us_ang_cor = round(us_ang_cor)
  if np.isnan(us_ang_cor):
    us_ang_cor = '---'

#Create Storm Relative Flow Based On Selected Data
if storm_motion_method == 'Mean Wind':
  sr_u, sr_v, sr_mw_u, sr_br_u, sr_bl_u, sr_mw_v, sr_br_v, sr_bl_v, sr_sfc_u, sr_sfc_v, sr_cu_u, sr_cd_u, sr_cd_v, sr_cu_v, sr_dtm_u, sr_dtm_v, sr_sm_u, sr_sm_v = calculate_sr_wind(u_mean, v_mean)

if storm_motion_method == 'User Selected':
  sr_u, sr_v, sr_mw_u, sr_br_u, sr_bl_u, sr_mw_v, sr_br_v, sr_bl_v, sr_sfc_u, sr_sfc_v, sr_cu_u, sr_cd_u, sr_cd_v, sr_cu_v, sr_dtm_u, sr_dtm_v, sr_sm_u, sr_sm_v = calculate_sr_wind(u_sm, v_sm)

if storm_motion_method == 'Bunkers Right':
  sr_u, sr_v, sr_mw_u, sr_br_u, sr_bl_u, sr_mw_v, sr_br_v, sr_bl_v, sr_sfc_u, sr_sfc_v, sr_cu_u, sr_cd_u, sr_cd_v, sr_cu_v, sr_dtm_u, sr_dtm_v, sr_sm_u, sr_sm_v = calculate_sr_wind(rmu, rmv)

if storm_motion_method == 'Bunkers Left':
    sr_u, sr_v, sr_mw_u, sr_br_u, sr_bl_u, sr_mw_v, sr_br_v, sr_bl_v, sr_sfc_u, sr_sfc_v, sr_cu_u, sr_cd_u, sr_cd_v, sr_cu_v, sr_dtm_u, sr_dtm_v, sr_sm_u, sr_sm_v = calculate_sr_wind(lmu, lmv)

if storm_motion_method == 'Corfidi Downshear':
    sr_u, sr_v, sr_mw_u, sr_br_u, sr_bl_u, sr_mw_v, sr_br_v, sr_bl_v, sr_sfc_u, sr_sfc_v, sr_cu_u, sr_cd_u, sr_cd_v, sr_cu_v, sr_dtm_u, sr_dtm_v, sr_sm_u, sr_sm_v = calculate_sr_wind(cvd_u, cvd_v)

if storm_motion_method == 'Corfidi Upshear':
    sr_u, sr_v, sr_mw_u, sr_br_u, sr_bl_u, sr_mw_v, sr_br_v, sr_bl_v, sr_sfc_u, sr_sfc_v, sr_cu_u, sr_cd_u, sr_cd_v, sr_cu_v, sr_dtm_u, sr_dtm_v, sr_sm_u, sr_sm_v = calculate_sr_wind(cvu_u, cvu_v)

#Calculate SRH from RM Motion
if max_height >= 500:
    SRH05 = (mpcalc.storm_relative_helicity(height = ALT_m * units.m, u = U_Wind_Knots * units.kts, v = V_Wind_Knots*units.kts, depth = 0.5*units.km, storm_u=rmu*units.kts, storm_v=rmv*units.kts))[0]
    if np.isnan(SRH05):
      SRH05 = '---'
    else:
     SRH05 = round(SRH05)
else:
  SRH05 = '---'

if max_height >= 1000:
  SRH1 = (mpcalc.storm_relative_helicity(height = ALT_m * units.m, u = U_Wind_Knots * units.kts, v = V_Wind_Knots*units.kts, depth = 1*units.km, storm_u=rmu*units.kts, storm_v=rmv*units.kts))[0]
  if np.isnan(SRH1):
    SRH1 = '---'
  else:
   SRH1 = round(SRH1)
else:
  SRH1 = '---'

if max_height >= 3000:
  SRH3 = (mpcalc.storm_relative_helicity(height = ALT_m * units.m, u = U_Wind_Knots * units.kts, v = V_Wind_Knots*units.kts, depth = 3*units.km, storm_u=rmu*units.kts, storm_v=rmv*units.kts))[0]
  if np.isnan(SRH3):
    SRH3 = '---'
  else:
    SRH3 = round(SRH3)

  SRH13 = (mpcalc.storm_relative_helicity(height = ALT_m * units.m, u = U_Wind_Knots * units.kts, v = V_Wind_Knots*units.kts, depth = 3*units.km, bottom = 1*units.km, storm_u=rmu*units.kts, storm_v=rmv*units.kts))[0]
  if np.isnan(SRH13):
    SRH13 = '---'
  else:
    SRH13 = round(SRH13)

else:
  SRH3 = '---'
  SRH13 = '---'

SRH_units = (units.m*units.m)/(units.s*units.s)
ureg=UnitRegistry()
try:
  SRH05=ureg(str(SRH05)).m
except:
  pass
try:
  SRH1=ureg(str(SRH1)).m
except:
  pass
try:
  SRH3=ureg(str(SRH3)).m
except:
  pass

#Calculate SR Wind
if max_height >= 500:
  SR_05U, SR_05V = calc_meanwind(sr_u, sr_v, 0,500)
  SR05 = calc_vector(SR_05U, SR_05V)[0]
  if np.isnan(SR05):
    SR05 = '--'
  else:
    SR05 = round(SR05)
else:
  SR05 = '--'

if max_height >= 1000:
  SR_1U, SR_1V = calc_meanwind(sr_u, sr_v, 0,1000)
  SR1 = calc_vector(SR_1U, SR_1V)[0]
  if np.isnan(SR1):
    SR1 = '--'
  else:
    SR1 = round(SR1)
else:
  SR1 = '--'

if max_height >= 3000:
  SR_3U, SR_3V = calc_meanwind(sr_u, sr_v, 0,3000)
  SR3 = calc_vector(SR_3U, SR_3V)[0]
  if np.isnan(SR3):
    SR3 = '--'
  else:
    SR3 = round(SR3)

  SR_13U, SR_13V = calc_meanwind(sr_u, sr_v, 1000,3000)
  SR13 = calc_vector(SR_13U, SR_13V)[0]
  if np.isnan(SR13):
    SR13 = '--'
  else:
    SR13 = round(SR13)
else:
  SR3 = '--'
  SR13 = '--'

if max_height >= 6000:
  SR_6U, SR_6V = calc_meanwind(sr_u, sr_v, 0,6000)
  SR6 = calc_vector(SR_6U, SR_6V)[0]
  if np.isnan(SR6):
    SR6 = '--'
  else:
    SR6 = round(SR6)

  SR_36U, SR_36V = calc_meanwind(sr_u, sr_v, 3000,6000)
  SR36 = calc_vector(SR_36U, SR_36V)[0]
  if np.isnan(SR36):
    SR36 = '--'
  else:
    SR36 = round(SR36)
else:
  SR6 = '--'
  SR36 = '--'

if max_height >= 8000:
  SR_8U, SR_8V = calc_meanwind(sr_u, sr_v, 0,8000)
  SR8 = calc_vector(SR_8U, SR_8V)[0]
  if np.isnan(SR8):
    SR8 = '--'
  else:
    SR8 = round(SR8)

  SR_68U, SR_68V = calc_meanwind(sr_u, sr_v, 6000,8000)
  SR68 = calc_vector(SR_68U, SR_68V)[0]
  if np.isnan(SR68):
    SR68 = '--'
  else:
    SR68 = round(SR68)
else:
  SR8 = '--'
  SR68 = '--'

if max_height >= 11000:
  SR_911U, SR_911V = calc_meanwind(sr_u, sr_v, 9000,11000)
  SR911 = calc_vector(SR_911U, SR_911V)[0]
  if np.isnan(SR911):
    SR911 = '--'
  else:
    SR911 = round(SR911)
else:
  SR911 = '--'

#Calculate Streamwise Vorticity

# adopted from Sam Brandt (2022)  and Kyle Gillett (2023)
# CONVERT TO m/s (uses `sm_u, sm_v` calculated above)
u_ms = (U_Wind_Knots/1.94384)
v_ms = (V_Wind_Knots/1.94384)
sm_u_ms = (sr_u/1.94384)
sm_v_ms = (sr_v/1.94384)

# INTEROPLATED SRW (send back to knots)
srw = mpcalc.wind_speed(sm_u_ms*units('m/s'), sm_v_ms*units('m/s'))
srw_knots = (srw.m*1.94384)

# SHEAR COMPONENTS FOR VORT CALC
# calc example = change in u over change in z
dudz = (u_ms[2::]-u_ms[0:-2]) / (ALT_m[2::]-ALT_m[0:-2])
dvdz = (v_ms[2::]-v_ms[0:-2]) / (ALT_m[2::]-ALT_m[0:-2])
dudz = np.insert(dudz,0,dudz[0])
dudz = np.insert(dudz,-1,dudz[-1])
dvdz = np.insert(dvdz,0,dvdz[0])
dvdz = np.insert(dvdz,-1,dvdz[-1])
# Shear magnitude,
shear=(np.sqrt(dudz**2+dvdz**2)+0.0000001)
# Vorticity components
uvort=-dvdz
vvort=dudz
# Total horizontal vorticity
totvort = np.sqrt(uvort**2 + vvort**2)
# Total streamwise vorticity
total_swvort = abs((sm_u_ms*uvort+sm_v_ms*vvort)/(np.sqrt(sm_u_ms**2+sm_v_ms**2)))
# Streamwiseness fraction
swvper = (total_swvort/shear)*100

# layer average streamwiseness and total streamwise vorticity
if max_height >= 500:
  swper05  = np.mean(swvper[0:5])
  swvort05       = np.mean(total_swvort[0:5])
  if np.isnan(swper05):
    swper05 = '--'
  else:
    swper05 = round(swper05)
  if np.isnan(swvort05):
    swvort05 = '---'
  else:
    swvort05 = round(swvort05, 3)
else:
  swper05 = '--'
  swvort05 = '---'

if max_height >= 1000:
  swper1 = np.mean(swvper[0:10])
  swvort1      = np.mean(total_swvort[0:10])
  if np.isnan(swper1):
    swper1 = '--'
  else:
    swper1 = round(swper1)
  if np.isnan(swvort1):
    swvort1 = '---'
  else:
    swvort1 = round(swvort1, 3)
else:
  swper1 = '--'
  swvort1 = '---'

if max_height >= 3000:
  swper3 = np.mean(swvper[0:30])
  swvort3      = np.mean(total_swvort[0:30])
  if np.isnan(swper3):
    swper3 = '--'
  else:
    swper3 = round(swper3)
  if np.isnan(swvort3):
    swvort3 = '---'
  else:
    swvort3 = round(swvort3, 3)
else:
  swper3 = '--'
  swvort3 = '---'

swvort_units = units.s**-1

sr_spd = calc_vector(sr_u, sr_v)[0]

#Plot Ground Relative Hodograph

In [ ]:
# @title
def round_up_nearest(n):
    return 5 * math.ceil(n / 5)

#Create Figure
fig = plt.figure(figsize=(16,9), facecolor='white', edgecolor="black", linewidth = 6)
ax=fig.add_subplot(1,1,1)
if range_type == 'Dynamic':
  #Determine Component Ring For Dynamic
  magar = []
  magar.append(SPD.max())
  magar.append(mean_mag)
  magar.append(rmag)
  magar.append(lmag)
  magar.append(cor_d_mag)
  magar.append(cor_u_mag)
  magar.append(dtm_mag)
  max2 = max(magar)
  hodo_rang = round_up_nearest(max2+10)
  h = Hodograph(ax, component_range = hodo_rang)
if range_type == 'Static':
  h = Hodograph(ax, component_range = static_value)
h.add_grid(increment = 10)

#Create Colormap
boundaries = np.array([0,1000,3000,6000,8000,12000, 16000])
colors = ['purple', 'red', 'green', 'gold', 'blue', 'teal']

#Plot Hodograph and Winds
l = h.plot_colormapped(U_Wind_Knots, V_Wind_Knots, ALT_m, intervals = boundaries, colors = colors)
try:
  mw = ax.scatter(u_mean, v_mean, color = 'darkorange', marker = 's', label = f"0-6km MW: {'{:.0f}'.format(mean_dirmet)}°/{'{:.0f} kt'.format(mean_mag)}", s = 125)
except:
  mw = ax.scatter(u_mean, v_mean, color = 'darkorange', marker = 's', label = f"0-6km MW: {mean_dirmet}°/{mean_mag} kt", s = 125)
try:
  rm = ax.scatter(rmu, rmv, color = 'red', marker = 'o', label = f"Bunkers RM: {'{:.0f}'.format(rang)}°/{'{:.0f} kt'.format(rmag)}", s = 125)
except:
  rm = ax.scatter(rmu, rmv, color = 'red', marker = 'o', label = f"Bunkers RM: {rang}°/{rmag} kt", s = 125)
try:
  lm = ax.scatter(lmu, lmv, color = 'blue', marker = 'o', label = f"Bunkers LM: {'{:.0f}'.format(lang)}°/{'{:.0f} kt'.format(lmag)}", s = 125)
except:
  lm = ax.scatter(lmu, lmv, color = 'blue', marker = 'o', label = f"Bunkers LM: {f'{lang}'}°/{f'{lmag} kt'}", s = 125)
try:
  cd = ax.scatter(cvd_u, cvd_v, color = 'deeppink', marker = 'd', s = 125, label = f"Corfidi DS: {'{:.0f}'.format(down_adj)}°/{'{:.0f} kt'.format(cor_d_mag)}")
except:
  cd = ax.scatter(cvd_u, cvd_v, color = 'deeppink', marker = 'd', s = 125, label = f"Corfidi DS: {f'{down_adj}°/{cor_d_mag} kt'}")
try:
  cu = ax.scatter(cvu_u, cvu_v, color = 'green', marker = 'd', s = 125, label = f"Corfidi US: {'{:.0f}'.format(up_adj)}°/{'{:.0f} kt'.format(cor_u_mag)}")
except:
  cu = ax.scatter(cvu_u, cvu_v, color = 'green', marker = 'd', s = 125, label = f"Corfidi US: {f'{up_adj}°/{cor_u_mag} kt'}")
try:
  dtm = ax.scatter(dtm_u, dtm_v, color = 'black', marker = 'v', s = 125, label = f"DTM: {'{:.0f}'.format(dtm_dir_cor)}°/{'{:.0f} kt'.format(dtm_mag)} ")
except:
  dtm = ax.scatter(dtm_u, dtm_v, color = 'black', marker = 'v', s = 125, label = f"DTM: {f'{dtm_dir_cor}°/{dtm_mag} kt'} ")

if storm_motion_method == 'User Selected':
    us = ax.scatter(u_sm, v_sm, color = 'black', marker = 'x', label = f"User SM: {'{:.0f}'.format(sm_dir)}/{'{:.0f}'.format(sm_speed)}", s = 125)
if sfc_status != 'None':
    sfc = ax.scatter(sfc_u, sfc_v, color = 'purple', marker = 'x', s = 85, label = f"Sfc. Wind: {'{:.0f}'.format(sfc_dir)}°/{'{:.0f} kt'.format(sfc_spd)}")
    plt.plot([sfc_u, U_Wind_Knots[0]], [sfc_v, V_Wind_Knots[0]], color="purple", linestyle = '--', linewidth = 2)

#Add Colorbar and Fig Text
CS = plt.colorbar(l, pad=0.00)
CS.set_label('Meters Above Radar')

plt.figtext(0.91, 0.9, "BS", fontsize = 14, weight = 'bold')
plt.figtext(0.95, 0.9, "SRW", fontsize = 14, weight = 'bold')
plt.figtext(0.995, 0.9, "SRH", fontsize = 14, weight = 'bold')
plt.figtext(1.055, 0.9, "SWζ%", fontsize = 14, weight = 'bold')
plt.figtext(1.11, 0.9, f"SWζ", fontsize = 14, weight = 'bold')

try:
  plt.figtext(0.85,0.87, f" 0-500m:", fontsize = 12, weight = 'bold', color = 'purple')
except:
  plt.figtext(0.85,0.87, f" 0-500m:", fontsize = 12, weight = 'bold', color = 'purple')
try:
  plt.figtext(0.91,0.87, f"{'{:.0f}'.format(shr005)} kt", fontsize = 12, weight = 'bold', color = 'purple')
except:
    plt.figtext(0.91,0.87, f"{shr005} kt", fontsize = 12, weight = 'bold', color = 'purple')
try:
  plt.figtext(0.95,0.87, f"{'{:.0f}'.format(SR05) } kt", fontsize = 12, weight = 'bold', color = 'purple')
except:
  plt.figtext(0.95,0.87, f"{SR05} kt", fontsize = 12, weight = 'bold', color = 'purple')

try:
  plt.figtext(0.99,0.87, f"{'{:.0f}'.format(SRH05) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'purple')
except:
  plt.figtext(0.99,0.87, f"{SRH05 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'purple')
try:
  plt.figtext(1.055,0.87, f"{'{:.0f}'.format(swper05) } %", fontsize = 12, weight = 'bold', color = 'purple')
except:
  plt.figtext(1.055,0.87, f"{swper05} %", fontsize = 12, weight = 'bold', color = 'purple')
try:
  plt.figtext(1.11,0.87, f"{'{:.3f}'.format(swvort05)} ", fontsize = 12, weight = 'bold', color = 'purple')
except:
  plt.figtext(1.11,0.87, f"{swvort05} ", fontsize = 12, weight = 'bold', color = 'purple')

try:
  plt.figtext(0.85,0.83, f" 0-1km: ", fontsize = 12, weight = 'bold', color = 'darkorchid')
except:
  plt.figtext(0.85,0.83, f" 0-1km: ", fontsize = 12, weight = 'bold', color = 'darkorchid')
try:
    plt.figtext(0.91,0.83, f"{'{:.0f}'.format(shr01)} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
except:
  plt.figtext(0.91,0.83, f"{shr01} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
try:
  plt.figtext(0.95,0.83, f"{'{:.0f}'.format(SR1) } kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
except:
  plt.figtext(0.95,0.83, f"{SR1} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
try:
  plt.figtext(0.99,0.83, f"{'{:.0f}'.format(SRH1) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkorchid')
except:
  plt.figtext(0.99,0.83, f"{SRH1 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkorchid')
try:
  plt.figtext(1.055,0.83, f"{'{:.0f}'.format(swper1) } %", fontsize = 12, weight = 'bold', color = 'darkorchid')
except:
  plt.figtext(1.055,0.83, f"{swper1} %", fontsize = 12, weight = 'bold', color = 'darkorchid')
try:
  plt.figtext(1.11,0.83, f"{'{:.3f}'.format(swvort1)} ", fontsize = 12, weight = 'bold', color = 'darkorchid')
except:
  plt.figtext(1.11,0.83, f"{swvort1} ", fontsize = 12, weight = 'bold', color = 'darkorchid')

try:
  plt.figtext(0.85,0.79, f" 0-3km: ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
except:
  plt.figtext(0.85,0.79, f" 0-3km: ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
try:
  plt.figtext(0.91,0.79, f"{'{:.0f}'.format(shr03)} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
except:
  plt.figtext(0.91,0.79, f"{shr03} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
try:
  plt.figtext(0.95,0.79, f"{'{:.0f}'.format(SR3) } kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
except:
  plt.figtext(0.95,0.79, f"{SR3} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
try:
  plt.figtext(0.99,0.79, f"{'{:.0f}'.format(SRH3) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumorchid')
except:
  plt.figtext(0.99,0.79, f"{SRH3 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumorchid')
try:
  plt.figtext(1.055,0.79, f"{'{:.0f}'.format(swper3) } %", fontsize = 12, weight = 'bold', color = 'mediumsorchid')
except:
  plt.figtext(1.055,0.79, f"{swper3} %", fontsize = 12, weight = 'bold', color = 'mediumorchid')
try:
  plt.figtext(1.11,0.79, f"{'{:.3f}'.format(swvort3)} ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
except:
  plt.figtext(1.11,0.79, f"{swvort3} ", fontsize = 12, weight = 'bold', color = 'mediumorchid')

try:
  plt.figtext(0.85,0.75, f" 0-6km: ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
except:
  plt.figtext(0.85,0.75, f" 0-6km: ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
try:
  plt.figtext(0.91,0.75, f"{'{:.0f}'.format(shr06)} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
except:
  plt.figtext(0.91,0.75, f"{shr06} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
try:
  plt.figtext(0.95,0.75, f"{'{:.0f}'.format(SR6) } kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
except:
  plt.figtext(0.95,0.75, f"{SR6} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')

plt.figtext(0.99,0.75, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
plt.figtext(1.055,0.75, f"-- %", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
plt.figtext(1.11,0.75, f" -- ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')

try:
  plt.figtext(0.85,0.71, f" 3-6km: ", fontsize = 12, weight = 'bold', color = 'mediumblue')
except:
  plt.figtext(0.85,0.71, f" 3-6km: ", fontsize = 12, weight = 'bold', color = 'mediumblue')
try:
  plt.figtext(0.91,0.71, f"{'{:.0f}'.format(shr36)} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
except:
  plt.figtext(0.91,0.71, f"{shr36} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
try:
  plt.figtext(0.95,0.71, f"{'{:.0f}'.format(SR36) } kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
except:
  plt.figtext(0.95,0.71, f"{SR36} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')

plt.figtext(0.99,0.71, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumblue')
plt.figtext(1.055,0.71, f"-- %", fontsize = 12, weight = 'bold', color = 'mediumblue')
plt.figtext(1.11,0.71, f" -- ", fontsize = 12, weight = 'bold', color = 'mediumblue')

try:
  plt.figtext(0.85,0.67, f" 0-8km: ", fontsize = 12, weight = 'bold', color = 'darkblue')
except:
  plt.figtext(0.85,0.67, f" 0-8km: ", fontsize = 12, weight = 'bold', color = 'darkblue')
try:
  plt.figtext(0.91,0.67, f"{'{:.0f}'.format(shr08)} kt", fontsize = 12, weight = 'bold', color = 'darkblue')
except:
  plt.figtext(0.91,0.67, f"{shr08} kt", fontsize = 12, weight = 'bold', color = 'darkblue')
try:
  plt.figtext(0.95,0.67, f"{'{:.0f}'.format(SR8) } kt", fontsize = 12, weight = 'bold', color = 'darkblue')
except:
  plt.figtext(0.95,0.67, f"{SR8} kt", fontsize = 12, weight = 'bold', color = 'darkblue')

plt.figtext(0.99,0.67, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkblue')
plt.figtext(1.055,0.67, f"-- %", fontsize = 12, weight = 'bold', color = 'darkblue')
plt.figtext(1.11,0.67, f" -- ", fontsize = 12, weight = 'bold', color = 'darkblue')

try:
  plt.figtext(0.85,0.63, f" 9-11km: ", fontsize = 12, weight = 'bold', color = 'midnightblue')
except:
  plt.figtext(0.85,0.63,f" 9-11km: ", fontsize = 12, weight = 'bold', color = 'midnightblue')
try:
  plt.figtext(0.91,0.63, f"{'{:.0f}'.format(shr911)} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
except:
  plt.figtext(0.91,0.63, f"{shr911} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
try:
  plt.figtext(0.95,0.63, f"{'{:.0f}'.format(SR911) } kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
except:
  plt.figtext(0.95,0.63, f"{SR911} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')

plt.figtext(0.99,0.63, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'midnightblue')
plt.figtext(1.055,0.63, f"-- %", fontsize = 12, weight = 'bold', color = 'midnightblue')
plt.figtext(1.11,0.63, f" -- ", fontsize = 12, weight = 'bold', color = 'midnightblue')

plt.figtext(0.90,0.585, "Storm Motion/Sfc Wind", fontsize = 14, weight = 'bold')

plt.legend(loc = 'right', bbox_to_anchor=(1.885, 0.535),
          ncol=2, fancybox=True, shadow=True, fontsize=11, facecolor='white', framealpha=1.0,
            labelcolor='k', borderpad=0.7)
plt.title(f'Hodograph from {radar_id} Valid {month}-{day}-{scan_time.year} {hour}:{minute}:{second} UTC', fontsize = 16, weight = 'bold')

try:
  #Plot SRW wrt Hgt
  sr_plot = plt.axes((0.895, 0.10, 0.095, 0.32))
  plt.figtext(0.94, 0.435, f'SR Wind (kts)', weight='bold', color='black', fontsize=12, ha='center')
  sr_plot.set_ylim(0,3000)
  sr_plot.set_xlim(sr_spd[0:31].min() -6,sr_spd[0:31].max() +6)
  sr_plot.plot(sr_spd[0:11], ALT_m[0:11], color = 'purple', linewidth = 3)
  sr_plot.plot(sr_spd[10:31], ALT_m[10:31], color = 'red', linewidth = 3)
  plt.ylabel('Height Above Radar (m)')
  plt.xlabel('SRW (kt)')
  plt.grid(axis='y')
except:
  pass

try:
  #Plot SW Vort Perc wrt Hgt
  swv_plot = plt.axes((1.05, 0.10, 0.095, 0.32))
  plt.figtext(1.1, 0.435, f'SWζ%', weight='bold', color='black', fontsize=12, ha='center')
  swv_plot.set_ylim(0,3000)
  swv_plot.set_xlim(00,101)
  swv_plot.plot(swvper[0:11], ALT_m[0:11], color = 'purple', linewidth = 3)
  swv_plot.plot(swvper[10:31], ALT_m[10:31], color = 'red', linewidth = 3)
  plt.ylabel('Height Above Radar (m)')
  plt.xlabel('SWζ%')
  plt.grid(axis='y')
except:
  pass
#Add Title and Legend and Save Figure

plt.savefig(f'/content/hodos/Hodograph_{radar_id}_{month}{day}{scan_time.year}_{hour}{minute}{second}.png', bbox_inches='tight')


#Plot Storm Relative Hodograph

In [ ]:
# @title
def round_up_nearest(n):
    return 5 * math.ceil(n / 5)

try:
    #Create Figure
    fig = plt.figure(figsize=(16,9), facecolor='white', edgecolor="black", linewidth = 6)
    ax=fig.add_subplot(1,1,1)
    if range_type == 'Dynamic':
      #Determine Component Ring For Dynamic
      magar = []
      magar.append(SPD.max())
      magar.append(mean_mag)
      magar.append(rmag)
      magar.append(lmag)
      magar.append(cor_d_mag)
      magar.append(cor_u_mag)
      magar.append(dtm_mag)
      max2 = max(magar)
      hodo_rang = round_up_nearest(max2+10)
      h = Hodograph(ax, component_range = hodo_rang)
    if range_type == 'Static':
      h = Hodograph(ax, component_range = static_value)
    h.add_grid(increment = 10)

    #Create Colormap
    boundaries = np.array([0,1000,3000,6000,8000,12000, 16000])
    colors = ['purple', 'red', 'green', 'gold', 'blue', 'teal']


    #Plot Hodograph and Winds
    l = h.plot_colormapped(sr_u, sr_v, ALT_m, intervals = boundaries, colors = colors)
    try:
      mw = ax.scatter(sr_mw_u, sr_mw_v, color = 'darkorange', marker = 's', label = f"0-6km MW: {'{:.0f}'.format(mean_dirmet)}°/{'{:.0f} kt'.format(mean_mag)}", s = 125)
    except:
      mw = ax.scatter(sr_mw_u, sr_mw_v, color = 'darkorange', marker = 's', label = f"0-6km MW: {mean_dirmet}°/{mean_mag} kt", s = 125)
    try:
      rm = ax.scatter(sr_br_u, sr_br_v, color = 'red', marker = 'o', label = f"Bunkers RM: {'{:.0f}'.format(rang)}°/{'{:.0f} kt'.format(rmag)}", s = 125)
    except:
      rm = ax.scatter(sr_br_u, sr_br_v, color = 'red', marker = 'o', label = f"Bunkers RM: {rang}°/{rmag} kt", s = 125)
    try:
      lm = ax.scatter(sr_bl_u, sr_bl_v, color = 'blue', marker = 'o', label = f"Bunkers LM: {'{:.0f}'.format(lang)}°/{'{:.0f} kt'.format(lmag)}", s = 125)
    except:
      lm = ax.scatter(sr_bl_u, sr_bl_v, color = 'blue', marker = 'o', label = f"Bunkers LM: {f'{lang}'}°/{f'{lmag} kt'}", s = 125)
    try:
      cd = ax.scatter(sr_cd_u, sr_cd_v, color = 'deeppink', marker = 'd', s = 125, label = f"Corfidi DS: {'{:.0f}'.format(down_adj)}°/{'{:.0f} kt'.format(cor_d_mag)}")
    except:
      cd = ax.scatter(sr_cd_u, sr_cd_v, color = 'deeppink', marker = 'd', s = 125, label = f"Corfidi DS: {f'{down_adj}°/{cor_d_mag} kt'}")
    try:
      cu = ax.scatter(sr_cu_u, sr_cu_v, color = 'green', marker = 'd', s = 125, label = f"Corfidi US: {'{:.0f}'.format(up_adj)}°/{'{:.0f} kt'.format(cor_u_mag)}")
    except:
      cu = ax.scatter(sr_cu_u, sr_cu_v, color = 'green', marker = 'd', s = 125, label = f"Corfidi US: {f'{up_adj}°/{cor_u_mag} kt'}")
    try:
      dtm = ax.scatter(sr_dtm_u, sr_dtm_v, color = 'black', marker = 'v', s = 125, label = f"DTM: {'{:.0f}'.format(dtm_dir_cor)}°/{'{:.0f} kt'.format(dtm_mag)} ")
    except:
      dtm = ax.scatter(sr_dtm_u, sr_dtm_v, color = 'black', marker = 'v', s = 125, label = f"DTM: {f'{dtm_dir_cor}°/{dtm_mag} kt'} ")

    if storm_motion_method == 'User Selected':
      us = ax.scatter(sr_sm_u, sr_sm_v, color = 'black', marker = 'x', label = f"User SM: {'{:.0f}'.format(sm_dir)}/{'{:.0f}'.format(sm_speed)}", s = 125)
    if sfc_status != 'None':
      sfc = ax.scatter(sr_sfc_u, sr_sfc_v, color = 'purple', marker = 'x', s = 85, label = f"Sfc. Wind: {'{:.0f}'.format(sfc_dir)}°/{'{:.0f} kt'.format(sfc_spd)}")
      plt.plot([sr_sfc_u, sr_u[0]], [sr_sfc_v, sr_v[0]], color="purple", linestyle = '--', linewidth = 2)

    #Add Colorbar and Fig Text
    CS = plt.colorbar(l, pad=0.00)
    CS.set_label('Meters Above Radar')

    plt.figtext(0.91, 0.9, "BS", fontsize = 14, weight = 'bold')
    plt.figtext(0.95, 0.9, "SRW", fontsize = 14, weight = 'bold')
    plt.figtext(0.995, 0.9, "SRH", fontsize = 14, weight = 'bold')
    plt.figtext(1.055, 0.9, "SWζ%", fontsize = 14, weight = 'bold')
    plt.figtext(1.11, 0.9, f"SWζ", fontsize = 14, weight = 'bold')

    try:
      plt.figtext(0.85,0.87, f" 0-500m:", fontsize = 12, weight = 'bold', color = 'purple')
    except:
      plt.figtext(0.85,0.87, f" 0-500m:", fontsize = 12, weight = 'bold', color = 'purple')
    try:
      plt.figtext(0.91,0.87, f"{'{:.0f}'.format(shr005)} kt", fontsize = 12, weight = 'bold', color = 'purple')
    except:
        plt.figtext(0.91,0.87, f"{shr005} kt", fontsize = 12, weight = 'bold', color = 'purple')
    try:
      plt.figtext(0.95,0.87, f"{'{:.0f}'.format(SR05) } kt", fontsize = 12, weight = 'bold', color = 'purple')
    except:
      plt.figtext(0.95,0.87, f"{SR05} kt", fontsize = 12, weight = 'bold', color = 'purple')

    try:
      plt.figtext(0.99,0.87, f"{'{:.0f}'.format(SRH05) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'purple')
    except:
      plt.figtext(0.99,0.87, f"{SRH05 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'purple')
    try:
      plt.figtext(1.055,0.87, f"{'{:.0f}'.format(swper05) } %", fontsize = 12, weight = 'bold', color = 'purple')
    except:
      plt.figtext(1.055,0.87, f"{swper05} %", fontsize = 12, weight = 'bold', color = 'purple')
    try:
      plt.figtext(1.11,0.87, f"{'{:.3f}'.format(swvort05)} ", fontsize = 12, weight = 'bold', color = 'purple')
    except:
      plt.figtext(1.11,0.87, f"{swvort05} ", fontsize = 12, weight = 'bold', color = 'purple')

    try:
      plt.figtext(0.85,0.83, f" 0-1km: ", fontsize = 12, weight = 'bold', color = 'darkorchid')
    except:
      plt.figtext(0.85,0.83, f" 0-1km: ", fontsize = 12, weight = 'bold', color = 'darkorchid')
    try:
        plt.figtext(0.91,0.83, f"{'{:.0f}'.format(shr01)} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
    except:
      plt.figtext(0.91,0.83, f"{shr01} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
    try:
      plt.figtext(0.95,0.83, f"{'{:.0f}'.format(SR1) } kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
    except:
      plt.figtext(0.95,0.83, f"{SR1} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
    try:
      plt.figtext(0.99,0.83, f"{'{:.0f}'.format(SRH1) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkorchid')
    except:
      plt.figtext(0.99,0.83, f"{SRH1 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkorchid')
    try:
      plt.figtext(1.055,0.83, f"{'{:.0f}'.format(swper1) } %", fontsize = 12, weight = 'bold', color = 'darkorchid')
    except:
      plt.figtext(1.055,0.83, f"{swper1} %", fontsize = 12, weight = 'bold', color = 'darkorchid')
    try:
      plt.figtext(1.11,0.83, f"{'{:.3f}'.format(swvort1)} ", fontsize = 12, weight = 'bold', color = 'darkorchid')
    except:
      plt.figtext(1.11,0.83, f"{swvort1} ", fontsize = 12, weight = 'bold', color = 'darkorchid')

    try:
      plt.figtext(0.85,0.79, f" 0-3km: ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    except:
      plt.figtext(0.85,0.79, f" 0-3km: ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    try:
      plt.figtext(0.91,0.79, f"{'{:.0f}'.format(shr03)} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    except:
      plt.figtext(0.91,0.79, f"{shr03} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    try:
      plt.figtext(0.95,0.79, f"{'{:.0f}'.format(SR3) } kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    except:
      plt.figtext(0.95,0.79, f"{SR3} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    try:
      plt.figtext(0.99,0.79, f"{'{:.0f}'.format(SRH3) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    except:
      plt.figtext(0.99,0.79, f"{SRH3 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    try:
      plt.figtext(1.055,0.79, f"{'{:.0f}'.format(swper3) } %", fontsize = 12, weight = 'bold', color = 'mediumsorchid')
    except:
      plt.figtext(1.055,0.79, f"{swper3} %", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    try:
      plt.figtext(1.11,0.79, f"{'{:.3f}'.format(swvort3)} ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
    except:
      plt.figtext(1.11,0.79, f"{swvort3} ", fontsize = 12, weight = 'bold', color = 'mediumorchid')

    try:
      plt.figtext(0.85,0.75, f" 0-6km: ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
    except:
      plt.figtext(0.85,0.75, f" 0-6km: ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
    try:
      plt.figtext(0.91,0.75, f"{'{:.0f}'.format(shr06)} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
    except:
      plt.figtext(0.91,0.75, f"{shr06} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
    try:
      plt.figtext(0.95,0.75, f"{'{:.0f}'.format(SR6) } kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
    except:
      plt.figtext(0.95,0.75, f"{SR6} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')

    plt.figtext(0.99,0.75, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
    plt.figtext(1.055,0.75, f"-- %", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
    plt.figtext(1.11,0.75, f" -- ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')

    try:
      plt.figtext(0.85,0.71, f" 3-6km: ", fontsize = 12, weight = 'bold', color = 'mediumblue')
    except:
      plt.figtext(0.85,0.71, f" 3-6km: ", fontsize = 12, weight = 'bold', color = 'mediumblue')
    try:
      plt.figtext(0.91,0.71, f"{'{:.0f}'.format(shr36)} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
    except:
      plt.figtext(0.91,0.71, f"{shr36} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
    try:
      plt.figtext(0.95,0.71, f"{'{:.0f}'.format(SR36) } kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
    except:
      plt.figtext(0.95,0.71, f"{SR36} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')

    plt.figtext(0.99,0.71, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumblue')
    plt.figtext(1.055,0.71, f"-- %", fontsize = 12, weight = 'bold', color = 'mediumblue')
    plt.figtext(1.11,0.71, f" -- ", fontsize = 12, weight = 'bold', color = 'mediumblue')

    try:
      plt.figtext(0.85,0.67, f" 0-8km: ", fontsize = 12, weight = 'bold', color = 'darkblue')
    except:
      plt.figtext(0.85,0.67, f" 0-8km: ", fontsize = 12, weight = 'bold', color = 'darkblue')
    try:
      plt.figtext(0.91,0.67, f"{'{:.0f}'.format(shr08)} kt", fontsize = 12, weight = 'bold', color = 'darkblue')
    except:
      plt.figtext(0.91,0.67, f"{shr08} kt", fontsize = 12, weight = 'bold', color = 'darkblue')
    try:
      plt.figtext(0.95,0.67, f"{'{:.0f}'.format(SR8) } kt", fontsize = 12, weight = 'bold', color = 'darkblue')
    except:
      plt.figtext(0.95,0.67, f"{SR8} kt", fontsize = 12, weight = 'bold', color = 'darkblue')

    plt.figtext(0.99,0.67, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkblue')
    plt.figtext(1.055,0.67, f"-- %", fontsize = 12, weight = 'bold', color = 'darkblue')
    plt.figtext(1.11,0.67, f" -- ", fontsize = 12, weight = 'bold', color = 'darkblue')

    try:
      plt.figtext(0.85,0.63, f" 9-11km: ", fontsize = 12, weight = 'bold', color = 'midnightblue')
    except:
      plt.figtext(0.85,0.63,f" 9-11km: ", fontsize = 12, weight = 'bold', color = 'midnightblue')
    try:
      plt.figtext(0.91,0.63, f"{'{:.0f}'.format(shr911)} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
    except:
      plt.figtext(0.91,0.63, f"{shr911} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
    try:
      plt.figtext(0.95,0.63, f"{'{:.0f}'.format(SR911) } kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
    except:
      plt.figtext(0.95,0.63, f"{SR911} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')

    plt.figtext(0.99,0.63, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'midnightblue')
    plt.figtext(1.055,0.63, f"-- %", fontsize = 12, weight = 'bold', color = 'midnightblue')
    plt.figtext(1.11,0.63, f" -- ", fontsize = 12, weight = 'bold', color = 'midnightblue')

    plt.figtext(0.90,0.595, "Storm Motion/Sfc Wind", fontsize = 14, weight = 'bold')

    plt.legend(loc = 'right', bbox_to_anchor=(1.885, 0.535),
              ncol=2, fancybox=True, shadow=True, fontsize=11, facecolor='white', framealpha=1.0,
                labelcolor='k', borderpad=0.7)
    plt.title(f'SR Hodograph from {radar_id} Valid {month}-{day}-{scan_time.year} {hour}:{minute}:{second} UTC', fontsize = 16, weight = 'bold')

    try:
    #Plot SRW wrt Hgt
      sr_plot = plt.axes((0.895, 0.10, 0.095, 0.32))
      plt.figtext(0.94, 0.435, f'SR Wind (kts)', weight='bold', color='black', fontsize=12, ha='center')
      sr_plot.set_ylim(0,3000)
      sr_plot.set_xlim(sr_spd[0:31].min() -6,sr_spd[0:31].max() +6)
      sr_plot.plot(sr_spd[0:11], ALT_m[0:11], color = 'purple', linewidth = 3)
      sr_plot.plot(sr_spd[10:31], ALT_m[10:31], color = 'red', linewidth = 3)
      plt.ylabel('Height Above Radar (m)')
      plt.xlabel('SRW (kt)')
      plt.grid(axis='y')
    except:
      pass

    try:
      #Plot SW Vort Perc wrt Hgt
      swv_plot = plt.axes((1.05, 0.10, 0.095, 0.32))
      plt.figtext(1.1, 0.435, f'SWζ%', weight='bold', color='black', fontsize=12, ha='center')
      swv_plot.set_ylim(0,3000)
      swv_plot.set_xlim(00,101)
      swv_plot.plot(swvper[0:11], ALT_m[0:11], color = 'purple', linewidth = 3)
      swv_plot.plot(swvper[10:31], ALT_m[10:31], color = 'red', linewidth = 3)
      plt.ylabel('Height Above Radar (m)')
      plt.xlabel('SWζ%')
      plt.grid(axis='y')
    except:
      pass

    plt.savefig(f'/content/sr_hodos/SR_Hodograph_{radar_id}_{month}{day}{scan_time.year}_{hour}{minute}{second}.png', bbox_inches='tight')

except:
    #Create Figure
  fig = plt.figure(figsize=(16,9), facecolor='white', edgecolor="black", linewidth = 6)
  ax=fig.add_subplot(1,1,1)
  if range_type == 'Dynamic':
    #Determine Component Ring For Dynamic
    magar = []
    magar.append(SPD.max())
    magar.append(mean_mag)
    magar.append(rmag)
    magar.append(lmag)
    magar.append(cor_d_mag)
    magar.append(cor_u_mag)
    magar.append(dtm_mag)
    max2 = max(magar)
    hodo_rang = round_up_nearest(max2+10)
    h = Hodograph(ax, component_range = hodo_rang)
  if range_type == 'Static':
    h = Hodograph(ax, component_range = static_value)
  h.add_grid(increment = 10)

  #Create Colormap
  boundaries = np.array([0,1000,3000,6000,8000,12000, 16000])
  colors = ['purple', 'red', 'green', 'gold', 'blue', 'teal']

  #Plot Hodograph and Winds
  l = h.plot_colormapped(U_Wind_Knots, V_Wind_Knots, ALT_m, intervals = boundaries, colors = colors)
  try:
    mw = ax.scatter(u_mean, v_mean, color = 'darkorange', marker = 's', label = f"0-6km MW: {'{:.0f}'.format(mean_dirmet)}°/{'{:.0f} kt'.format(mean_mag)}", s = 125)
  except:
    mw = ax.scatter(u_mean, v_mean, color = 'darkorange', marker = 's', label = f"0-6km MW: {mean_dirmet}°/{mean_mag} kt", s = 125)
  try:
    rm = ax.scatter(rmu, rmv, color = 'red', marker = 'o', label = f"Bunkers RM: {'{:.0f}'.format(rang)}°/{'{:.0f} kt'.format(rmag)}", s = 125)
  except:
    rm = ax.scatter(rmu, rmv, color = 'red', marker = 'o', label = f"Bunkers RM: {rang}°/{rmag} kt", s = 125)
  try:
    lm = ax.scatter(lmu, lmv, color = 'blue', marker = 'o', label = f"Bunkers LM: {'{:.0f}'.format(lang)}°/{'{:.0f} kt'.format(lmag)}", s = 125)
  except:
    lm = ax.scatter(lmu, lmv, color = 'blue', marker = 'o', label = f"Bunkers LM: {f'{lang}'}°/{f'{lmag} kt'}", s = 125)
  try:
    cd = ax.scatter(cvd_u, cvd_v, color = 'deeppink', marker = 'd', s = 125, label = f"Corfidi DS: {'{:.0f}'.format(down_adj)}°/{'{:.0f} kt'.format(cor_d_mag)}")
  except:
    cd = ax.scatter(cvd_u, cvd_v, color = 'deeppink', marker = 'd', s = 125, label = f"Corfidi DS: {f'{down_adj}°/{cor_d_mag} kt'}")
  try:
    cu = ax.scatter(cvu_u, cvu_v, color = 'green', marker = 'd', s = 125, label = f"Corfidi US: {'{:.0f}'.format(up_adj)}°/{'{:.0f} kt'.format(cor_u_mag)}")
  except:
    cu = ax.scatter(cvu_u, cvu_v, color = 'green', marker = 'd', s = 125, label = f"Corfidi US: {f'{up_adj}°/{cor_u_mag} kt'}")
  try:
    dtm = ax.scatter(dtm_u, dtm_v, color = 'black', marker = 'v', s = 125, label = f"DTM: {'{:.0f}'.format(dtm_dir_cor)}°/{'{:.0f} kt'.format(dtm_mag)} ")
  except:
    dtm = ax.scatter(dtm_u, dtm_v, color = 'black', marker = 'v', s = 125, label = f"DTM: {f'{dtm_dir_cor}°/{dtm_mag} kt'} ")

  if storm_motion_method == 'User Selected':
    us = ax.scatter(u_sm, v_sm, color = 'black', marker = 'x', label = f"User SM: {'{:.0f}'.format(sm_dir)}/{'{:.0f}'.format(sm_speed)}", s = 125)
  if sfc_status != 'None':
    sfc = ax.scatter(sfc_u, sfc_v, color = 'purple', marker = 'x', s = 85, label = f"Sfc. Wind: {'{:.0f}'.format(sfc_dir)}°/{'{:.0f} kt'.format(sfc_spd)}")
    plt.plot([sfc_u, U_Wind_Knots[0]], [sfc_v, V_Wind_Knots[0]], color="purple", linestyle = '--', linewidth = 2)

  #Add Colorbar and Fig Text
  CS = plt.colorbar(l, pad=0.00)
  CS.set_label('Meters Above Radar')

  plt.figtext(0.91, 0.9, "BS", fontsize = 14, weight = 'bold')
  plt.figtext(0.95, 0.9, "SRW", fontsize = 14, weight = 'bold')
  plt.figtext(0.995, 0.9, "SRH", fontsize = 14, weight = 'bold')
  plt.figtext(1.055, 0.9, "SWζ%", fontsize = 14, weight = 'bold')
  plt.figtext(1.11, 0.9, f"SWζ", fontsize = 14, weight = 'bold')

  try:
    plt.figtext(0.85,0.87, f" 0-500m:", fontsize = 12, weight = 'bold', color = 'purple')
  except:
    plt.figtext(0.85,0.87, f" 0-500m:", fontsize = 12, weight = 'bold', color = 'purple')
  try:
    plt.figtext(0.91,0.87, f"{'{:.0f}'.format(shr005)} kt", fontsize = 12, weight = 'bold', color = 'purple')
  except:
      plt.figtext(0.91,0.87, f"{shr005} kt", fontsize = 12, weight = 'bold', color = 'purple')
  try:
    plt.figtext(0.95,0.87, f"{'{:.0f}'.format(SR05) } kt", fontsize = 12, weight = 'bold', color = 'purple')
  except:
    plt.figtext(0.95,0.87, f"{SR05} kt", fontsize = 12, weight = 'bold', color = 'purple')

  try:
    plt.figtext(0.99,0.87, f"{'{:.0f}'.format(SRH05) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'purple')
  except:
    plt.figtext(0.99,0.87, f"{SRH05 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'purple')
  try:
    plt.figtext(1.055,0.87, f"{'{:.0f}'.format(swper05) } %", fontsize = 12, weight = 'bold', color = 'purple')
  except:
    plt.figtext(1.055,0.87, f"{swper05} %", fontsize = 12, weight = 'bold', color = 'purple')
  try:
    plt.figtext(1.11,0.87, f"{'{:.3f}'.format(swvort05)} ", fontsize = 12, weight = 'bold', color = 'purple')
  except:
    plt.figtext(1.11,0.87, f"{swvort05} ", fontsize = 12, weight = 'bold', color = 'purple')

  try:
    plt.figtext(0.85,0.83, f" 0-1km: ", fontsize = 12, weight = 'bold', color = 'darkorchid')
  except:
    plt.figtext(0.85,0.83, f" 0-1km: ", fontsize = 12, weight = 'bold', color = 'darkorchid')
  try:
      plt.figtext(0.91,0.83, f"{'{:.0f}'.format(shr01)} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
  except:
    plt.figtext(0.91,0.83, f"{shr01} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
  try:
    plt.figtext(0.95,0.83, f"{'{:.0f}'.format(SR1) } kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
  except:
    plt.figtext(0.95,0.83, f"{SR1} kt", fontsize = 12, weight = 'bold', color = 'darkorchid')
  try:
    plt.figtext(0.99,0.83, f"{'{:.0f}'.format(SRH1) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkorchid')
  except:
    plt.figtext(0.99,0.83, f"{SRH1 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkorchid')
  try:
    plt.figtext(1.055,0.83, f"{'{:.0f}'.format(swper1) } %", fontsize = 12, weight = 'bold', color = 'darkorchid')
  except:
    plt.figtext(1.055,0.83, f"{swper1} %", fontsize = 12, weight = 'bold', color = 'darkorchid')
  try:
    plt.figtext(1.11,0.83, f"{'{:.3f}'.format(swvort1)} ", fontsize = 12, weight = 'bold', color = 'darkorchid')
  except:
    plt.figtext(1.11,0.83, f"{swvort1} ", fontsize = 12, weight = 'bold', color = 'darkorchid')

  try:
    plt.figtext(0.85,0.79, f" 0-3km: ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  except:
    plt.figtext(0.85,0.79, f" 0-3km: ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  try:
    plt.figtext(0.91,0.79, f"{'{:.0f}'.format(shr03)} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  except:
    plt.figtext(0.91,0.79, f"{shr03} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  try:
    plt.figtext(0.95,0.79, f"{'{:.0f}'.format(SR3) } kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  except:
    plt.figtext(0.95,0.79, f"{SR3} kt", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  try:
    plt.figtext(0.99,0.79, f"{'{:.0f}'.format(SRH3) * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  except:
    plt.figtext(0.99,0.79, f"{SRH3 * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  try:
    plt.figtext(1.055,0.79, f"{'{:.0f}'.format(swper3) } %", fontsize = 12, weight = 'bold', color = 'mediumsorchid')
  except:
    plt.figtext(1.055,0.79, f"{swper3} %", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  try:
    plt.figtext(1.11,0.79, f"{'{:.3f}'.format(swvort3)} ", fontsize = 12, weight = 'bold', color = 'mediumorchid')
  except:
    plt.figtext(1.11,0.79, f"{swvort3} ", fontsize = 12, weight = 'bold', color = 'mediumorchid')

  try:
    plt.figtext(0.85,0.75, f" 0-6km: ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
  except:
    plt.figtext(0.85,0.75, f" 0-6km: ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
  try:
    plt.figtext(0.91,0.75, f"{'{:.0f}'.format(shr06)} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
  except:
    plt.figtext(0.91,0.75, f"{shr06} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
  try:
    plt.figtext(0.95,0.75, f"{'{:.0f}'.format(SR6) } kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
  except:
    plt.figtext(0.95,0.75, f"{SR6} kt", fontsize = 12, weight = 'bold', color = 'mediumslateblue')

  plt.figtext(0.99,0.75, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
  plt.figtext(1.055,0.75, f"-- %", fontsize = 12, weight = 'bold', color = 'mediumslateblue')
  plt.figtext(1.11,0.75, f" -- ", fontsize = 12, weight = 'bold', color = 'mediumslateblue')

  try:
    plt.figtext(0.85,0.71, f" 3-6km: ", fontsize = 12, weight = 'bold', color = 'mediumblue')
  except:
    plt.figtext(0.85,0.71, f" 3-6km: ", fontsize = 12, weight = 'bold', color = 'mediumblue')
  try:
    plt.figtext(0.91,0.71, f"{'{:.0f}'.format(shr36)} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
  except:
    plt.figtext(0.91,0.71, f"{shr36} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
  try:
    plt.figtext(0.95,0.71, f"{'{:.0f}'.format(SR36) } kt", fontsize = 12, weight = 'bold', color = 'mediumblue')
  except:
    plt.figtext(0.95,0.71, f"{SR36} kt", fontsize = 12, weight = 'bold', color = 'mediumblue')

  plt.figtext(0.99,0.71, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'mediumblue')
  plt.figtext(1.055,0.71, f"-- %", fontsize = 12, weight = 'bold', color = 'mediumblue')
  plt.figtext(1.11,0.71, f" -- ", fontsize = 12, weight = 'bold', color = 'mediumblue')

  try:
    plt.figtext(0.85,0.67, f" 0-8km: ", fontsize = 12, weight = 'bold', color = 'darkblue')
  except:
    plt.figtext(0.85,0.67, f" 0-8km: ", fontsize = 12, weight = 'bold', color = 'darkblue')
  try:
    plt.figtext(0.91,0.67, f"{'{:.0f}'.format(shr08)} kt", fontsize = 12, weight = 'bold', color = 'darkblue')
  except:
    plt.figtext(0.91,0.67, f"{shr08} kt", fontsize = 12, weight = 'bold', color = 'darkblue')
  try:
    plt.figtext(0.95,0.67, f"{'{:.0f}'.format(SR8) } kt", fontsize = 12, weight = 'bold', color = 'darkblue')
  except:
    plt.figtext(0.95,0.67, f"{SR8} kt", fontsize = 12, weight = 'bold', color = 'darkblue')

  plt.figtext(0.99,0.67, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'darkblue')
  plt.figtext(1.055,0.67, f"-- %", fontsize = 12, weight = 'bold', color = 'darkblue')
  plt.figtext(1.11,0.67, f" -- ", fontsize = 12, weight = 'bold', color = 'darkblue')

  try:
    plt.figtext(0.85,0.63, f" 9-11km: ", fontsize = 12, weight = 'bold', color = 'midnightblue')
  except:
    plt.figtext(0.85,0.63,f" 9-11km: ", fontsize = 12, weight = 'bold', color = 'midnightblue')
  try:
    plt.figtext(0.91,0.63, f"{'{:.0f}'.format(shr911)} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
  except:
    plt.figtext(0.91,0.63, f"{shr911} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
  try:
    plt.figtext(0.95,0.63, f"{'{:.0f}'.format(SR911) } kt", fontsize = 12, weight = 'bold', color = 'midnightblue')
  except:
    plt.figtext(0.95,0.63, f"{SR911} kt", fontsize = 12, weight = 'bold', color = 'midnightblue')

  plt.figtext(0.99,0.63, f"{'---' * SRH_units:~P}", fontsize = 12, weight = 'bold', color = 'midnightblue')
  plt.figtext(1.055,0.63, f"-- %", fontsize = 12, weight = 'bold', color = 'midnightblue')
  plt.figtext(1.11,0.63, f" -- ", fontsize = 12, weight = 'bold', color = 'midnightblue')

  plt.figtext(0.90,0.585, "Storm Motion/Sfc Wind", fontsize = 14, weight = 'bold')

  plt.legend(loc = 'right', bbox_to_anchor=(1.885, 0.535),
            ncol=2, fancybox=True, shadow=True, fontsize=11, facecolor='white', framealpha=1.0,
              labelcolor='k', borderpad=0.7)
  plt.title(f'Hodograph from {radar_id} Valid {month}-{day}-{scan_time.year} {hour}:{minute}:{second} UTC', fontsize = 16, weight = 'bold')

  try:
    #Plot SRW wrt Hgt
    sr_plot = plt.axes((0.895, 0.10, 0.095, 0.32))
    plt.figtext(0.94, 0.435, f'SR Wind (kts)', weight='bold', color='black', fontsize=12, ha='center')
    sr_plot.set_ylim(0,3000)
    sr_plot.set_xlim(sr_spd[0:31].min() -6,sr_spd[0:31].max() +6)
    sr_plot.plot(sr_spd[0:11], ALT_m[0:11], color = 'purple', linewidth = 3)
    sr_plot.plot(sr_spd[10:31], ALT_m[10:31], color = 'red', linewidth = 3)
    plt.ylabel('Height Above Radar (m)')
    plt.xlabel('SRW (kt)')
    plt.grid(axis='y')
  except:
    pass

  try:
    #Plot SW Vort Perc wrt Hgt
    swv_plot = plt.axes((1.05, 0.10, 0.095, 0.32))
    plt.figtext(1.1, 0.435, f'SWζ%', weight='bold', color='black', fontsize=12, ha='center')
    swv_plot.set_ylim(0,3000)
    swv_plot.set_xlim(00,101)
    swv_plot.plot(swvper[0:11], ALT_m[0:11], color = 'purple', linewidth = 3)
    swv_plot.plot(swvper[10:31], ALT_m[10:31], color = 'red', linewidth = 3)
    plt.ylabel('Height Above Radar (m)')
    plt.xlabel('SWζ%')
    plt.grid(axis='y')
  except:
    pass
  #Add Title and Legend and Save Figure
  plt.figtext(0.55, 0.75, f'SR Hodograph Unavailable \n Due to Poor Data Quality', weight='bold', color='red', fontsize=24, ha='center')

  plt.savefig(f'/content/sr_hodos/SR_Hodograph_{radar_id}_{month}{day}{scan_time.year}_{hour}{minute}{second}.png', bbox_inches='tight')

#Add Title and Legend and Save Figure
del sfc_u, sfc_v, sfc_dir
print(f"Hodographs for {radar_id} Valid {month}-{day}-{scan_time.year} {hour}:{minute}:{second} UTC Complete")